In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='319ce38c-200b-4e4b-91bf-4a46a0c8c70d', project_access_token='p-fb744bb2ce502dd3534deab78d7295226faa9438')
pc = project.project_context


# Predecir el abandono de clientes (Churn) de una Telco utilizando SparkML en IBM Cloud Pak for Data (ICP4D)

Usaremos este cuaderno para crear un modelo de aprendizaje automático para predecir el abandono de clientes. En este cuaderno, crearemos el modelo de predicción utilizando la biblioteca SparkML.

Este cuaderno lo guía a través de estos pasos:

- Cargar y visualizar un data set. (https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv)
- Crear un modelo predictivo con SparkML API
- Guardar el modelo en un repositorio de Ml

In [ ]:

# El token del proyecto es un token de autorización que se utiliza para acceder a los recursos del proyecto, como fuentes de datos, conexiones y que utilizan las API de la plataforma.
# Genere el TOKEN en la sesión de configuración e inserte aquí el código usando el menú de arriba (3 puntos) "Insertar Token del Proyecto"


## 1.0 Instalar los paquetes requeridos

Hay un par de paquetes de Python que usaremos en este cuaderno. Primero nos aseguramos de que se elimine el cliente Watson Machine Learning v3 (no está instalado por defecto) y luego instalamos / actualizamos la versión v4 del cliente (este paquete se instala por defecto en CP4D).

WML Client: https://wml-api-pyclient-dev-v4.mybluemix.net/#repository

In [2]:
!pip uninstall --yes watson-machine-learning-client-V4
!pip install --user watson-machine-learning-client-V4
!pip install --user pyspark==2.4 --upgrade|tail -n 1
!pip install --user scikit-learn==0.20.3 --upgrade|tail -n 1

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Found existing installation: watson-machine-learning-client-V4 1.0.103
Uninstalling watson-machine-learning-client-V4-1.0.103:
  Successfully uninstalled watson-machine-learning-client-V4-1.0.103
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_

In [3]:
import pandas as pd
import numpy as np
import json
import os
import warnings

warnings.filterwarnings("ignore")

## 2.0 Carga y limpia de datos

Vamos a cargar nuestros datos como un dataframe de Pandas.

**<font color='red'><< SIGUE LAS INSTRUCCIONES DE ABAJO PARA CARGAR EL DATASET >></font>**

* Resalte la celda de abajo haciendo clic en ella.
* Haga clic en el icono "Buscar datos" `10/01` en la esquina superior derecha del cuaderno.
* Si está utilizando datos virtualizados, comience eligiendo la pestaña `Archivos`. Luego, elija sus datos virtualizados (es decir, MYSCHEMA.BILLINGPRODUCTCUSTOMERS), haga clic en `Insertar en el código` y elija` Insertar Pandas DataFrame`.
* Si está utilizando este cuaderno sin datos virtualizados, agregue el archivo cargado localmente `Telco-Customer-Churn.csv` eligiendo la pestaña` Archivos`. Luego elija el `Telco-Customer-Churn.csv`. Haga clic en `Insertar en el código` y elija` Insertar Pandas DataFrame`.
* El código para llevar los datos al entorno del portátil y crear un Pandas DataFrame se agregará a la celda a continuación.
* Ejecute la celda

In [ ]:
# Resalta esta celda e ingresa el DataFrame de Pandas para los datos de los clientes
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# Inserta el código de pandas abajo de esta línea.


Usaremos la convención de nomenclatura de Pandas "df" para nuestro DataFrame. Asegúrese de que la celda de abajo use el nombre del marco de datos usado arriba. Para el archivo cargado localmente, debería verse como df_data_1 o df_data_2 o df_data_x. Para el caso de datos virtualizados, debería verse como data_df_1 o data_df_2 o data_df_x.

**<font color='red'><< ACTUALIZAR LA ASIGNACIÓN DE VARIABLE A LA VARIABLE GENERADA ANTERIORMENTE. >></font>**

In [ ]:
# for virtualized data
# df = data_df_1

# for local upload
df = df_data_2

### 2.1 Eliminar la característica CustomerID (columna)

In [ ]:
df = df.drop('customerID', axis=1)

In [ ]:
df.head()

### 2.2 Examina los tipos de datos de las características

In [ ]:
df.info()

In [ ]:
# Estadísticas de las columnas (características). Configúrelo en todos, ya que el valor predeterminado es describir solo las funciones numéricas.
df.describe(include = 'all')

Vemos que la tenencia varía de 0 (nuevo cliente) a 6 años, los cargos mensuales varían de $ 18 a $ 118, etc.

### 2.3 Verifique la necesidad de convertir la columna TotalCharges a numérico si esta es detectada como un objeto.

Si el `df.info` anterior muestra la columna" TotalCharges "como un objeto, necesitaremos convertirlo a numérico. Si ya hizo esto durante un ejercicio anterior de "Visualización de datos con refinería de datos", puede saltar al paso `2.4`.

In [ ]:
totalCharges = df.columns.get_loc("TotalCharges")
new_col = pd.to_numeric(df.iloc[:, totalCharges], errors='coerce')
df.iloc[:, totalCharges] = pd.Series(new_col)

In [ ]:
# Estadísticas de las columnas (características). Configúrelo en todos, ya que el valor predeterminado es describir solo las funciones numéricas.
df.describe(include = 'all')

Ahora vemos estadísticas para la función `TotalCharges`.

### 2.4 Cualquier valor NaN debe de ser removido para crear un modelo más preciso.

In [ ]:
# Verifique si tenemos valores de NaN y vea qué características tienen valores faltantes que deben abordarse
print(df.isnull().values.any())
df.isnull().sum()

Deberíamos ver que a la columna `TotalCharges` le faltan valores. Hay varias formas de abordar este problema:

- Eliminar registros con valores perdidos
- Complete el valor faltante con una de las siguientes estrategias: Cero, Media de los valores de la columna, Valor aleatorio, etc.).

In [ ]:
# Manejar valores perdidos para la columna nan_column (TotalCharges)
from sklearn.impute import SimpleImputer

# Encuentra el número de columna para TotalCharges (comenzando en 0).
total_charges_idx = df.columns.get_loc("TotalCharges")
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

df.iloc[:, total_charges_idx] = imputer.fit_transform(df.iloc[:, total_charges_idx].values.reshape(-1, 1))
df.iloc[:, total_charges_idx] = pd.Series(df.iloc[:, total_charges_idx])

In [ ]:
# Validar que nos hemos hecho cargo de cualquier valor NaN
print(df.isnull().values.any())
df.isnull().sum()


### 2.5 Clasificar las características.

Clasificaremos algunas de las columnas / características en función de si son valores categóricos o valores continuos (es decir, numéricos). Usaremos esto en secciones posteriores para crear visualizaciones.

In [ ]:
columns_idx = np.s_[0:] # Fragmento de la primera fila(header) con todas las columnas.
first_record_idx = np.s_[0] # Índice de el primer registro

string_fields = [type(fld) is str for fld in df.iloc[first_record_idx, columns_idx]] # Todos los campos tipo string
all_features = [x for x in df.columns if x != 'Churn']
categorical_columns = list(np.array(df.columns)[columns_idx][string_fields])
categorical_features = [x for x in categorical_columns if x != 'Churn']
continuous_features = [x for x in all_features if x not in categorical_features]

#print('All Features: ', all_features)
#print('\nCategorical Features: ', categorical_features)
#print('\nContinuous Features: ', continuous_features)
#print('\nAll Categorical Columns: ', categorical_columns)

### 2.6 Visualizar los datos

La visualización de datos se puede utilizar para encontrar patrones, detectar valores atípicos, comprender la distribución y más. Podemos utilizar gráficos como:

- Histogramas, diagramas de caja, etc: Para encontrar la distribución / propagación de nuestras variables continuas.
- Gráficos de barras: para mostrar la frecuencia en valores categóricos.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

%matplotlib inline
sns.set(style="darkgrid")
sns.set_palette("hls", 3)

Primero, obtenemos una vista de alto nivel de la distribución de la "Tasa de abandono (Churn)". ¿Qué porcentaje de clientes en nuestro conjunto de datos están agitando frente a los que no lo están?

In [ ]:
print(df.groupby(['Churn']).size())
churn_plot = sns.countplot(data=df, x='Churn', order=df.Churn.value_counts().index)
plt.ylabel('Count')
for p in churn_plot.patches:
    height = p.get_height()
    churn_plot.text(p.get_x()+p.get_width()/2., height + 1,'{0:.0%}'.format(height/float(len(df))),ha="center") 
plt.show()

Podemos utilizar gráficos de recuentos de frecuencia para comprender las características categóricas relativas a la "Tasa de abandono (Churn)"

- Podemos ver eso para la función de género. Tenemos tasas de abandono relativamente iguales por "género"
- Podemos ver eso para la función `InternetService`. Tenemos una mayor tasa de abandono para los que tienen servicio de "fibra óptica" en comparación con los que tienen "DSL".

In [ ]:
# Gráficos de recuento de características categóricas.
f, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9), (ax10, ax11, ax12), (ax13, ax14, ax15)) = plt.subplots(5, 3, figsize=(20, 20))
ax = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12, ax13, ax14, ax15 ]

for i in range(len(categorical_features)):
    sns.countplot(x = categorical_features[i], hue="Churn", data=df, ax=ax[i])

Podemos usar gráficos de histrograma para comprender la distribución de nuestras características continuas / numéricas en relación con el abandono.

- Podemos ver que para la función `MonthlyCharges`, los clientes que abandonan tienden a pagar tarifas mensuales más altas que los que se quedan.
- Podemos ver que para la función de tenencia, los clientes que abandonan tienden a ser clientes relativamente nuevos.

In [ ]:
# Histogramas de características continuas.
fig, ax = plt.subplots(2, 2, figsize=(28, 8))
df[df.Churn == 'No'][continuous_features].hist(bins=20, color="blue", alpha=0.5, ax=ax)
df[df.Churn == 'Yes'][continuous_features].hist(bins=20, color="orange", alpha=0.5, ax=ax)

# O usa displots
#sns.set_palette("hls", 3)
#f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(25, 25))
#ax = [ax1, ax2, ax3, ax4]
#for i in range(len(continuous_features)):
#    sns.distplot(df[continuous_features[i]], bins=20, hist=True, ax=ax[i])

In [ ]:
# Crea una cuadrícula para relaciones por pares
gr = sns.PairGrid(df, height=5, hue="Churn")
gr = gr.map_diag(plt.hist)
gr = gr.map_offdiag(plt.scatter)
gr = gr.add_legend()

In [ ]:
# Trace diagramas de caja de columnas numéricas. Una mayor variación en la gráfica de caja implica una mayor importancia.
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(25, 25))
ax = [ax1, ax2, ax3, ax4]

for i in range(len(continuous_features)):
    sns.boxplot(x = 'Churn', y = continuous_features[i], data=df, ax=ax[i])

## 3.0 Creación de un modelo de ML

Ahora podemos crear nuestro modelo de aprendizaje automático. Puede usar la información / intuición obtenida de los pasos de visualización de datos anteriores para saber qué tipo de modelo crear o qué funciones usar. Crearemos un modelo de clasificación simple.

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.createDataFrame(df)
df_data.head()

### 3.1 Divida los datos en conjuntos de prueba y entrenamiento

In [ ]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

print("Número de registros para el entrenamiento: " + str(train_data.count()))
print("Número de registros para la evaluación: " + str(test_data.count()))

### 3.2 Examina el esquema del DataFrame Spark
Observe los tipos de datos para determinar los requisitos para la ingeniería de funciones.

In [ ]:
spark_df.printSchema()

### 3.3 Utiliza la función StringIndexer para codificar una columna de cadenas de etiquetas en una columna de índices de etiquetas

Estamos utilizando el paquete Pipeline para construir los pasos de desarrollo como una pipeline.
Estamos usando StringIndexer para manejar características categóricas / de cadena del conjunto de datos. StringIndexer codifica una columna de cadenas de etiquetas en una columna de índices de etiquetas

Luego usamos VectorAssembler para ensamblar estas características en un vector. La API de canalizaciones requiere que las variables de entrada se pasen en un vector

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model


si_gender = StringIndexer(inputCol = 'gender', outputCol = 'gender_IX')
si_Partner = StringIndexer(inputCol = 'Partner', outputCol = 'Partner_IX')
si_Dependents = StringIndexer(inputCol = 'Dependents', outputCol = 'Dependents_IX')
si_PhoneService = StringIndexer(inputCol = 'PhoneService', outputCol = 'PhoneService_IX')
si_MultipleLines = StringIndexer(inputCol = 'MultipleLines', outputCol = 'MultipleLines_IX')
si_InternetService = StringIndexer(inputCol = 'InternetService', outputCol = 'InternetService_IX')
si_OnlineSecurity = StringIndexer(inputCol = 'OnlineSecurity', outputCol = 'OnlineSecurity_IX')
si_OnlineBackup = StringIndexer(inputCol = 'OnlineBackup', outputCol = 'OnlineBackup_IX')
si_DeviceProtection = StringIndexer(inputCol = 'DeviceProtection', outputCol = 'DeviceProtection_IX')
si_TechSupport = StringIndexer(inputCol = 'TechSupport', outputCol = 'TechSupport_IX')
si_StreamingTV = StringIndexer(inputCol = 'StreamingTV', outputCol = 'StreamingTV_IX')
si_StreamingMovies = StringIndexer(inputCol = 'StreamingMovies', outputCol = 'StreamingMovies_IX')
si_Contract = StringIndexer(inputCol = 'Contract', outputCol = 'Contract_IX')
si_PaperlessBilling = StringIndexer(inputCol = 'PaperlessBilling', outputCol = 'PaperlessBilling_IX')
si_PaymentMethod = StringIndexer(inputCol = 'PaymentMethod', outputCol = 'PaymentMethod_IX')


In [ ]:
si_Label = StringIndexer(inputCol="Churn", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

### 3.4 Creando un solo vector

In [ ]:
va_features = VectorAssembler(inputCols=['gender_IX',  'SeniorCitizen', 'Partner_IX', 'Dependents_IX', 'PhoneService_IX', 'MultipleLines_IX', 'InternetService_IX', \
                                         'OnlineSecurity_IX', 'OnlineBackup_IX', 'DeviceProtection_IX', 'TechSupport_IX', 'StreamingTV_IX', 'StreamingMovies_IX', \
                                         'Contract_IX', 'PaperlessBilling_IX', 'PaymentMethod_IX', 'TotalCharges', 'MonthlyCharges'], outputCol="features")

### 3.5 Crear una pipeline, y ajustar un modelo usando un algoritmo RandomForestClassifier 
Reúna todas las etapas en una pipeline. No esperamos una regresión lineal limpia, por lo que usaremos RandomForestClassifier para encontrar el mejor árbol de decisión para los datos.

In [ ]:
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_gender, si_Partner, si_Dependents, si_PhoneService, si_MultipleLines, si_InternetService, si_OnlineSecurity, si_OnlineBackup, si_DeviceProtection, \
                            si_TechSupport, si_StreamingTV, si_StreamingMovies, si_Contract, si_PaperlessBilling, si_PaymentMethod, si_Label, va_features, \
                            classifier, label_converter])

model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderROC')
area_under_curve = evaluatorDT.evaluate(predictions)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderPR')
area_under_PR = evaluatorDT.evaluate(predictions)
print("areaUnderROC = %g" % area_under_curve)

## 4.0 Guarda el modelo y prueba los datos.

Ahora el modelo se puede guardar para una futura implementación. El modelo se guardará utilizando el cliente Watson Machine Learning, en un espacio de implementación.
**<font color='red'><< ACTUALIZAR LA VARIABLE 'MODEL_NAME' A UN NOMBRE ÚNICO>></font>**

**<font color='red'><< ACTUALIZAR LA VARIABLE 'DEPLOYMENT_SPACE_NAME' AL NOMBRE DEL ESPACIO DE DESPLIEGUE CREADO ANTERIORMENTE >></font>**

In [ ]:

MODEL_NAME = "PREDICT-CHURN"
DEPLOYMENT_SPACE_NAME = 'Big-Data'


### 4.1 Guarda el modelo en una instancia local de ICP4D Watson Machine Learning

1. Genera una API Key: https://cloud.ibm.com/iam/apikeys
2. Genera un TOKEN para tu instancia de Watson Machine Learning:
   curl --insecure -X POST --header "Content-Type: application/x-www-form-urlencoded" --header "Accept: application/json" --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" --data-urlencode "apikey=$API_key" "https://iam.ng.bluemix.net/identity/token"
3.   <font color='red'>Reemplaza el valor de `token` marcado con `*****` con el `token` generado. El valor de el campo `url` debe ser igual a el valor del campo `url` de tu instancia de Watson Machine Learning.</font>

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "token":"*******"
}

client = APIClient(wml_credentials)

In [ ]:
client.spaces.list()

### Utiliza el espacio deseado como el `default_space`

El ID del espacio de implementación se buscará según el nombre especificado anteriormente. Si no recibe un GUID de espacio como resultado de la siguiente celda, no continúe hasta que haya creado un espacio de implementación.

In [ ]:
# Asegúrese de actualizar el nombre del espacio con el que desea utilizar.
#client.spaces.list()
all_spaces = client.spaces.get_details()['resources']
space_id = None
#print(all_spaces)
for space in all_spaces:
    if space['entity']['name'] == DEPLOYMENT_SPACE_NAME:
        space_id = space["metadata"]["id"]
        print("\nDeployment Space GUID: ", space_id)

if space_id is None:
    print("ADVERTENCIA: Tu espacio no existe. Cree un espacio de implementación antes de pasar a la siguiente celda.")
    #space_id = client.spaces.store(meta_props={client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]

**<font color='red'><< REEMPLAZA space_id de ABAJO con el ID de tu espacio. Por ejemplo: <br/>client.set.default_space("6b39c537-f707-4078-9dc7-ce70b70ab22f") >></font>**

In [ ]:
# Ahora configure el espacio predeterminado en el GUID para su espacio de implementación. Si tiene éxito, verá un mensaje de "Success".
client.set.default_space(space_id)

#### Guarda el Modelo

In [ ]:
# En caso de que necesite verificar los servicios, descomente la línea a continuación y ejecútelo.
#client.software_specifications.list()

In [ ]:
software_spec_id =  client.software_specifications.get_id_by_name('spark-mllib_2.4')
print(software_spec_id)

In [ ]:
# Guarda el modelo.
model_props = {client.repository.ModelMetaNames.NAME: MODEL_NAME,
               client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : software_spec_id,
               client.repository.ModelMetaNames.TYPE : "mllib_2.4"}
published_model = client.repository.store_model(model=model, pipeline=pipeline, meta_props=model_props, training_data=train_data)

print(json.dumps(published_model, indent=3))

In [ ]:
# Utilice esta celda para realizar cualquier limpieza de modelos e implementaciones previamente creados
client.repository.list_models()
client.deployments.list()

# client.repository.delete('GUID de el modelo guardado')
# client.deployments.delete('GUID de el modelo desplegado')


## 5.0 Guarda los datos de pruebas

Guardaremos los datos de prueba que usamos para evaluar el modelo en nuestro proyecto.

In [ ]:
write_score_CSV=test_data.toPandas().drop(['Churn'], axis=1)
#write_score_CSV.to_csv('/project_data/data_asset/TelcoCustomerSparkMLBatchScore.csv', sep=',', index=False)
project.save_data('TelcoCustomerSparkMLBatchScore.csv', write_score_CSV.to_csv(), overwrite=True)

write_eval_CSV=test_data.toPandas()
#write_eval_CSV.to_csv('/project_data/data_asset/TelcoCustomerSparkMLEval.csv', sep=',', index=False)
project.save_data('TelcoCustomerSparkMLEval.csv', write_eval_CSV.to_csv(), overwrite=True)

## Felicidades!!! ha creado un modelo basado en datos de abandono de clientes y lo ha implementado en Watson Machine Learning.